In [5]:
pip install optunity


Note: you may need to restart the kernel to use updated packages.


In [1]:
## LSSVM 参数寻优
import optunity
@optunity.cross_validated(x=data, y=labels, num_folds=5)
def nested_cv(x_train, y_train, x_test, y_test):
 
    # inner cross-validation to estimate performance of a set of hyperparameters
    @optunity.cross_validated(x=x_train, y=y_train, num_folds=10, num_iter=2)
    def inner_cv(x_train, y_train, x_test, y_test, C, gamma):


        RR2_SVM,RMSE_SVM,MAPE_SVM,RR2_train,RMSE_train,MAPE_train,svm_y_hat,svm_y_hat_train = LSSVM(train_x,train_y,test_x,test_y,C,gamma)
        
        return RMSE_SVM # RR2_SVM

    hpars, info, _ = optunity.minimize(inner_cv, num_evals=20,
                                    C=[0, 1000], gamma=[2**-15, 1])
    print('')
    print('Hyperparameters: ' + str(hpars))
    print('Cross-validated R2 after tuning: %1.3f' % info.optimum)
    print(hpars['C'],hpars['gamma'])
    
    RR2_SVM,RMSE_SVM,MAPE_SVM,RR2_train,RMSE_train,MAPE_train,svm_y_hat,svm_y_hat_train = LSSVM(train_x,train_y,test_x,test_y,hpars['C'],hpars['gamma'])

    return RMSE_SVM # RR2_SVM

# RMSE = nested_cv()
# print('')
# print('Nested RMSE: %1.3f' % RMSE)
RR2_SVM = nested_cv()
print('')
print('LSSVM Nested R2: %1.3f' % RR2_SVM)



NameError: name 'data' is not defined

In [43]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 28 03:18:22 2018


@author: HAN_RUIZHI yb77447@umac.mo OR  501248792@qq.com

This code is the first version of BLS Python. 
If you have any questions about the code or find any bugs
   or errors during use, please feel free to contact me.
If you have any questions about the original paper, 
   please contact the authors of related paper.
"""
import numpy as np
from sklearn import preprocessing
from numpy import random
import time
from sklearn import metrics

def tansig(x):
    return (2/(1+np.exp(-2*x)))-1

def pinv(A,reg):
    return np.mat(reg*np.eye(A.shape[1])+A.T.dot(A)).I.dot(A.T)


'''
参数压缩
'''
def shrinkage(a,b):
    z = np.maximum(a - b, 0) - np.maximum( -a - b, 0)
    return z
'''
参数稀疏化
'''
def sparse_bls(A,b):
    lam = 0.001
    itrs = 50
    AA = np.dot(A.T,A)   
    m = A.shape[1]
    n = b.shape[1]
    wk = np.zeros([m,n],dtype = 'double')
    ok = np.zeros([m,n],dtype = 'double')
    uk = np.zeros([m,n],dtype = 'double')
    L1 = np.mat(AA + np.eye(m)).I
    L2 = np.dot(np.dot(L1,A.T),b)
    for i in range(itrs):
        tempc = ok - uk
        ck =  L2 + np.dot(L1,tempc)
        ok = shrinkage(ck + uk, lam)
        uk += ck - ok
        wk = ok
    return wk
def bls_regression(train_x,train_y,test_x,test_y,s,C,NumFea,NumWin,NumEnhan):
    
    NumFea=int(NumFea)
    NumWin=int(NumWin)
    NumEnhan=int(NumEnhan)
   
    u = 0
    WF = list()
    for i in range(NumWin):
        random.seed(i+u)
        WeightFea=2*random.randn(train_x.shape[1]+1,NumFea)-1;
        WF.append(WeightFea)
#    random.seed(100)
    WeightEnhan=2*random.randn(NumWin*NumFea+1,NumEnhan)-1;
    time_start = time.time()
    H1 = np.hstack([train_x, 0.1 * np.ones([train_x.shape[0],1])]);
    y = np.zeros([train_x.shape[0],NumWin*NumFea])
    WFSparse = list()
    distOfMaxAndMin = np.zeros(NumWin)
    meanOfEachWindow = np.zeros(NumWin)
    for i in range(NumWin):
        WeightFea = WF[i]
        A1 = H1.dot(WeightFea)        
        scaler1 = preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit(A1)
        A1 = scaler1.transform(A1)
        WeightFeaSparse  = sparse_bls(A1,H1).T
        WFSparse.append(WeightFeaSparse)
    
        T1 = H1.dot(WeightFeaSparse)
        meanOfEachWindow[i] = T1.mean()
        distOfMaxAndMin[i] = T1.max() - T1.min()
        T1 = (T1 - meanOfEachWindow[i])/distOfMaxAndMin[i] 
        y[:,NumFea*i:NumFea*(i+1)] = T1

    H2 = np.hstack([y,0.1 * np.ones([y.shape[0],1])])
    T2 = H2.dot(WeightEnhan)
    T2 = tansig(T2);
    T3 = np.hstack([y,T2])
    WeightTop = pinv(T3,C).dot(train_y)

    Training_time = time.time()- time_start
    print('Training has been finished!');
    print('The Total Training Time is : ', round(Training_time,6), ' seconds' )
    NetoutTrain = T3.dot(WeightTop)

    RMSE = np.sqrt((NetoutTrain-train_y).T*(NetoutTrain-train_y)/train_y.shape[0]) #均方根误差
    MAPE = sum(abs(NetoutTrain-train_y))/train_y.mean()/train_y.shape[0]  #平均绝对百分比误差
    RR2=metrics.r2_score(NetoutTrain,train_y)#决定系数
    
    train_ERR = RMSE
    train_MAPE = MAPE
    train_RR2=RR2
    
    print('Training RMSE is : ',RMSE);
    print('Training MAPE is : ', MAPE)
    print('Training RR2 is : ',RR2)
    
    time_start = time.time()
    HH1 = np.hstack([test_x, 0.1 * np.ones([test_x.shape[0],1])])
    yy1=np.zeros([test_x.shape[0],NumWin*NumFea])
    for i in range(NumWin):
        WeightFeaSparse = WFSparse[i]
        TT1 = HH1.dot(WeightFeaSparse)
        TT1  = (TT1 - meanOfEachWindow[i])/distOfMaxAndMin[i]   
        yy1[:,NumFea*i:NumFea*(i+1)] = TT1

    HH2 = np.hstack([yy1, 0.1 * np.ones([yy1.shape[0],1])])
    TT2 = tansig(HH2.dot( WeightEnhan));
    TT3 = np.hstack([yy1,TT2])
    NetoutTest = TT3.dot(WeightTop)
    RMSE = np.sqrt((NetoutTest-test_y).T*(NetoutTest-test_y)/test_y.shape[0]);
    MAPE = sum(abs(NetoutTest-test_y))/test_y.mean()/test_y.shape[0]
    RR2=metrics.r2_score(NetoutTest,test_y)
    
    test_ERR = RMSE
    test_MAPE = MAPE
    test_RR2=RR2
    
#%% Calculate the testing accuracy
    Testing_time = time.time() - time_start
    print('Testing has been finished!');
    print('The Total Testing Time is : ', round(Testing_time,6), ' seconds' );
    print('Testing RMSE is : ', RMSE) 
    print('Testing MAPE is : ', MAPE)
    print('Testing RR2 is : ',RR2)
    return test_ERR,test_MAPE,test_RR2,Testing_time,train_ERR,train_MAPE,train_RR2,Training_time



In [16]:
import pandas as pd

datatest=pd.read_csv(r"C:\Users\ZHUO\Desktop\test_fsMIR_scaler.csv",index_col=0)
datatest.shape #1365,177

datatrain=pd.read_csv(r"C:\Users\ZHUO\Desktop\train_fsMIR_scaler.csv",index_col=0)
#datatrain.shape  #3182,177

datatrain.head()

,rest_time,heart_rate_stop,heart_rate_start,time_elapsed,total_running_time,time_since_last_stop,time_from_start_to_stop,altitude_start,distance_start,speed_start,...,avg_temp2,avg_temp_mean,avg_lat,avg_long,avg_alt,avg_hr,avg_speed,total_dist,avg_temp_mode,heart_rate_change_value
0,0.000000,0.586592,0.150,0.011415,0.011415,0.097215,0.011415,0.531499,0.000000,0.202312,...,0.806452,0.789631,0.734097,0.003288,0.648744,0.484522,0.241328,0.309353,0.806452,0.699605
1,0.021614,0.223464,0.465,0.058189,0.056588,0.047849,0.003341,0.496353,0.073158,0.349470,...,0.919355,0.915345,0.735361,0.003270,0.650670,0.665217,0.213962,0.134174,0.935484,0.193676
2,0.002642,0.592179,0.470,0.099603,0.096680,0.164782,0.072040,0.565650,0.043598,0.149446,...,0.596774,0.560446,0.735529,0.003270,0.666284,0.646265,0.350605,0.124125,0.596774,0.450593
3,0.000720,0.452514,0.520,0.155495,0.149231,0.202703,0.022064,0.503647,0.207981,0.339355,...,0.645161,0.597356,0.734828,0.003279,0.604050,0.563850,0.251099,0.189119,0.645161,0.312253
4,0.012968,0.636872,0.200,0.108095,0.090903,0.130723,0.006891,0.528183,0.113962,0.165222,...,0.370968,0.296575,0.736166,0.003266,0.649857,0.348246,0.100705,0.160134,0.370968,0.695652


In [21]:
datatest.head()

,rest_time,heart_rate_stop,heart_rate_start,time_elapsed,total_running_time,time_since_last_stop,time_from_start_to_stop,altitude_start,distance_start,speed_start,...,avg_temp2,avg_temp_mean,avg_lat,avg_long,avg_alt,avg_hr,avg_speed,total_dist,avg_temp_mode,heart_rate_change_value
0,0.006724,0.486034,0.345,0.166214,0.162108,0.219250,0.049210,0.539456,0.163111,0.001084,...,0.838710,0.812975,0.735712,0.003189,0.659856,0.406908,0.133332,0.171063,0.838710,0.474308
1,0.018492,0.368715,0.240,0.044199,0.029094,0.019443,0.000974,0.580570,0.045203,0.005660,...,0.677419,0.641900,0.735557,0.003239,0.696870,0.633106,0.339680,0.038229,0.677419,0.474308
2,0.005764,0.625698,0.560,0.236305,0.159950,0.013376,0.002645,0.535809,0.295905,0.006744,...,0.693548,0.652492,0.735470,0.003278,0.674507,0.476990,0.356060,0.269436,0.693548,0.403162
3,0.002642,0.486034,0.470,0.096819,0.093826,0.124931,0.002158,0.566313,0.156488,0.000000,...,0.806452,0.794795,0.735548,0.003270,0.678807,0.514625,0.304041,0.124154,0.822581,0.375494
4,0.006244,0.765363,0.340,0.010162,0.010162,0.019443,0.006264,0.528515,0.004126,0.000000,...,0.596774,0.543685,0.735544,0.003270,0.679090,0.833955,0.454427,0.123098,0.596774,0.675889


In [20]:
result=datatest.append(datatrain)
result.shape  #4547,177
result.head()

,rest_time,heart_rate_stop,heart_rate_start,time_elapsed,total_running_time,time_since_last_stop,time_from_start_to_stop,altitude_start,distance_start,speed_start,...,avg_temp2,avg_temp_mean,avg_lat,avg_long,avg_alt,avg_hr,avg_speed,total_dist,avg_temp_mode,heart_rate_change_value
0,0.006724,0.486034,0.345,0.166214,0.162108,0.219250,0.049210,0.539456,0.163111,0.001084,...,0.838710,0.812975,0.735712,0.003189,0.659856,0.406908,0.133332,0.171063,0.838710,0.474308
1,0.018492,0.368715,0.240,0.044199,0.029094,0.019443,0.000974,0.580570,0.045203,0.005660,...,0.677419,0.641900,0.735557,0.003239,0.696870,0.633106,0.339680,0.038229,0.677419,0.474308
2,0.005764,0.625698,0.560,0.236305,0.159950,0.013376,0.002645,0.535809,0.295905,0.006744,...,0.693548,0.652492,0.735470,0.003278,0.674507,0.476990,0.356060,0.269436,0.693548,0.403162
3,0.002642,0.486034,0.470,0.096819,0.093826,0.124931,0.002158,0.566313,0.156488,0.000000,...,0.806452,0.794795,0.735548,0.003270,0.678807,0.514625,0.304041,0.124154,0.822581,0.375494
4,0.006244,0.765363,0.340,0.010162,0.010162,0.019443,0.006264,0.528515,0.004126,0.000000,...,0.596774,0.543685,0.735544,0.003270,0.679090,0.833955,0.454427,0.123098,0.596774,0.675889


In [41]:
data=np.array(result.iloc[:,:-1])
labels=np.array(result.iloc[:,-1]).reshape(-1,1)
labels


array([[0.0067243 , 0.48603352, 0.345     , ..., 0.13333194, 0.1710633 ,
        0.83870968],
       [0.01849183, 0.36871508, 0.24      , ..., 0.33967992, 0.03822948,
        0.67741935],
       [0.00576369, 0.62569832, 0.56      , ..., 0.35606014, 0.26943576,
        0.69354839],
       ...,
       [0.00744476, 0.57541899, 0.3       , ..., 0.21075853, 0.36894795,
        0.82258065],
       [0.00552354, 0.4972067 , 0.38      , ..., 0.29634896, 0.38342039,
        0.66129032],
       [0.0129683 , 0.61452514, 0.43      , ..., 0.50641969, 0.27301111,
        0.51612903]])

In [50]:
## BLS 参数寻优
import optunity

@optunity.cross_validated(x=data, y=labels, num_folds=5)
def nested_cv(x_train, y_train, x_test, y_test):

    # inner cross-validation to estimate performance of a set of hyperparameters  train_x,train_y,test_x,test_y,s,C,NumFea,NumWin,NumEnhan
    @optunity.cross_validated(x=x_train, y=y_train, num_folds=10, num_iter=2)
    def inner_cv(x_train, y_train, x_test, y_test,NumFea,NumWin,NumEnhan):


        test_ERR,test_MAPE,test_RR2,Testing_time,train_ERR,train_MAPE,train_RR2,Training_time = bls_regression(x_train, y_train, x_test, y_test,0.8,2**-30,NumFea,NumWin,NumEnhan)
        
        return test_ERR # RR2_SVM

    hpars, info, _ = optunity.minimize(inner_cv, num_evals=20,NumFea=[1,20],NumWin=[1,30],NumEnhan=[1,200])
                                   # C=[0, 1000], gamma=[2**-15, 1])
    print('')
    print('Hyperparameters: ' + str(hpars))
    print('Cross-validated R2 after tuning: %1.3f' % info.optimum)
    #print(hpars['C'],hpars['gamma'])
    print(0.8,2**-30,hpars['NumFea'],hpars['NumWin'],hpars['NumEnhan'])
    
    test_ERR,test_MAPE,test_RR2,Testing_time,train_ERR,train_MAPE,train_RR2,Training_time = bls_regression(x_train, y_train, x_test, y_test,0.8,2**-30,hpars['NumFea'],hpars['NumWin'],hpars['NumEnhan'])
    #RR2_SVM,RMSE_SVM,MAPE_SVM,RR2_train,RMSE_train,MAPE_train,svm_y_hat,svm_y_hat_train = LSSVM(train_x,train_y,test_x,test_y,hpars['C'],hpars['gamma'])

    return test_ERR # RR2_SVM

RMSE = nested_cv()
print('')
print('Nested RMSE: %1.3f' % RMSE)
# RR2_SVM = nested_cv()
# print('')
# print('LSSVM Nested R2: %1.3f' % RR2_SVM)



Training has been finished!
The Total Training Time is :  0.250608  seconds
Training RMSE is :  [[0.00124471]]
Training MAPE is :  [[0.00235787]]
Training RR2 is :  0.9998856504478046
Testing has been finished!
The Total Testing Time is :  0.006358  seconds
Testing RMSE is :  [[0.01060656]]
Testing MAPE is :  [[0.00332898]]
Testing RR2 is :  0.9915743534366767
Training has been finished!
The Total Training Time is :  0.253292  seconds
Training RMSE is :  [[0.00084554]]
Training MAPE is :  [[0.00134809]]
Training RR2 is :  0.9999475429345736
Testing has been finished!
The Total Testing Time is :  0.005985  seconds
Testing RMSE is :  [[0.00100324]]
Testing MAPE is :  [[0.00152915]]
Testing RR2 is :  0.9999184798880438
Training has been finished!
The Total Training Time is :  0.252367  seconds
Training RMSE is :  [[0.00054208]]
Training MAPE is :  [[0.00072918]]
Training RR2 is :  0.9999782411268606
Testing has been finished!
The Total Testing Time is :  0.005876  seconds
Testing RMSE is 

Training RMSE is :  [[0.00962735]]
Training MAPE is :  [[0.01331666]]
Training RR2 is :  0.9930619537437639
Testing has been finished!
The Total Testing Time is :  0.00437  seconds
Testing RMSE is :  [[0.01000963]]
Testing MAPE is :  [[0.01402174]]
Testing RR2 is :  0.992558198901843
Training has been finished!
The Total Training Time is :  0.037899  seconds
Training RMSE is :  [[0.00969573]]
Training MAPE is :  [[0.01339709]]
Training RR2 is :  0.9929246918235248
Testing has been finished!
The Total Testing Time is :  0.003793  seconds
Testing RMSE is :  [[0.00866763]]
Testing MAPE is :  [[0.01220865]]
Testing RR2 is :  0.9947731256014163
Training has been finished!
The Total Training Time is :  0.036904  seconds
Training RMSE is :  [[0.00971895]]
Training MAPE is :  [[0.01344197]]
Training RR2 is :  0.9929333161358669
Testing has been finished!
The Total Testing Time is :  0.002992  seconds
Testing RMSE is :  [[0.00953918]]
Testing MAPE is :  [[0.01356589]]
Testing RR2 is :  0.993240

Training RMSE is :  [[0.02032826]]
Training MAPE is :  [[0.02933791]]
Training RR2 is :  0.9678205530409328
Testing has been finished!
The Total Testing Time is :  0.004033  seconds
Testing RMSE is :  [[0.02562499]]
Testing MAPE is :  [[0.03503799]]
Testing RR2 is :  0.9493218033080858
Training has been finished!
The Total Training Time is :  0.084775  seconds
Training RMSE is :  [[0.02066635]]
Training MAPE is :  [[0.02952485]]
Training RR2 is :  0.9666559701084095
Testing has been finished!
The Total Testing Time is :  0.003989  seconds
Testing RMSE is :  [[0.02350077]]
Testing MAPE is :  [[0.03297752]]
Testing RR2 is :  0.9583138334453045
Training has been finished!
The Total Training Time is :  0.08278  seconds
Training RMSE is :  [[0.02208002]]
Training MAPE is :  [[0.03151914]]
Training RR2 is :  0.9616255943276062
Testing has been finished!
The Total Testing Time is :  0.003988  seconds
Testing RMSE is :  [[0.02681961]]
Testing MAPE is :  [[0.03350588]]
Testing RR2 is :  0.94946

Training has been finished!
The Total Training Time is :  0.046877  seconds
Training RMSE is :  [[0.09285297]]
Training MAPE is :  [[0.14703732]]
Training RR2 is :  -0.799273380711055
Testing has been finished!
The Total Testing Time is :  0.003989  seconds
Testing RMSE is :  [[3.96247589]]
Testing MAPE is :  [[0.57572133]]
Testing RR2 is :  -0.0076044238422134924
Training has been finished!
The Total Training Time is :  0.04652  seconds
Training RMSE is :  [[0.08856738]]
Training MAPE is :  [[0.13957912]]
Training RR2 is :  -0.3597832585988918
Testing has been finished!
The Total Testing Time is :  0.004559  seconds
Testing RMSE is :  [[0.08793347]]
Testing MAPE is :  [[0.13533209]]
Testing RR2 is :  -0.339060651657886
Training has been finished!
The Total Training Time is :  0.04591  seconds
Training RMSE is :  [[0.08861113]]
Training MAPE is :  [[0.13989562]]
Training RR2 is :  -0.41305497454959417
Testing has been finished!
The Total Testing Time is :  0.003631  seconds
Testing RMS

Training has been finished!
The Total Training Time is :  0.155095  seconds
Training RMSE is :  [[0.00115737]]
Training MAPE is :  [[0.00201858]]
Training RR2 is :  0.9999015189081433
Testing has been finished!
The Total Testing Time is :  0.004382  seconds
Testing RMSE is :  [[0.00119351]]
Testing MAPE is :  [[0.00207348]]
Testing RR2 is :  0.9998900518451281
Training has been finished!
The Total Training Time is :  0.139579  seconds
Training RMSE is :  [[0.00286168]]
Training MAPE is :  [[0.00531377]]
Training RR2 is :  0.9993993034741462
Testing has been finished!
The Total Testing Time is :  0.006922  seconds
Testing RMSE is :  [[0.00292617]]
Testing MAPE is :  [[0.00547537]]
Testing RR2 is :  0.9993199055869487
Training has been finished!
The Total Training Time is :  0.110706  seconds
Training RMSE is :  [[0.00049497]]
Training MAPE is :  [[0.00084563]]
Training RR2 is :  0.9999819474382057
Testing has been finished!
The Total Testing Time is :  0.004717  seconds
Testing RMSE is 

...

Training has been finished!
The Total Training Time is :  0.136637  seconds
Training RMSE is :  [[0.00049891]]
Training MAPE is :  [[0.00081585]]
Training RR2 is :  0.9999806567036948
Testing has been finished!
The Total Testing Time is :  0.005984  seconds
Testing RMSE is :  [[0.00106543]]
Testing MAPE is :  [[0.00091832]]
Testing RR2 is :  0.9999187260370765
Training has been finished!
The Total Training Time is :  0.157612  seconds
Training RMSE is :  [[0.00075072]]
Training MAPE is :  [[0.00126383]]
Training RR2 is :  0.9999563298019434
Testing has been finished!
The Total Testing Time is :  0.005984  seconds
Testing RMSE is :  [[0.00077665]]
Testing MAPE is :  [[0.00129468]]
Testing RR2 is :  0.9999563311968285
Training has been finished!
The Total Training Time is :  0.170845  seconds
Training RMSE is :  [[0.00130506]]
Training MAPE is :  [[0.00233348]]
Training RR2 is :  0.9998687988036462
Testing has been finished!
The Total Testing Time is :  0.00798  seconds
Testing RMSE is :

Testing has been finished!
The Total Testing Time is :  0.008341  seconds
Testing RMSE is :  [[0.00118686]]
Testing MAPE is :  [[0.00219516]]
Testing RR2 is :  0.9998852706541326
Training has been finished!
The Total Training Time is :  0.149677  seconds
Training RMSE is :  [[0.0018674]]
Training MAPE is :  [[0.00315375]]
Training RR2 is :  0.9997347384420108
Testing has been finished!
The Total Testing Time is :  0.006982  seconds
Testing RMSE is :  [[0.00192503]]
Testing MAPE is :  [[0.00339003]]
Testing RR2 is :  0.9997018638677049
Training has been finished!
The Total Training Time is :  0.148603  seconds
Training RMSE is :  [[0.00172235]]
Training MAPE is :  [[0.0032843]]
Training RR2 is :  0.9997654007701865
Testing has been finished!
The Total Testing Time is :  0.006981  seconds
Testing RMSE is :  [[0.00185421]]
Testing MAPE is :  [[0.00328002]]
Testing RR2 is :  0.9997797125807281
Training has been finished!
The Total Training Time is :  0.150597  seconds
Training RMSE is :  [

Training has been finished!
The Total Training Time is :  0.107746  seconds
Training RMSE is :  [[0.00331841]]
Training MAPE is :  [[0.00471823]]
Training RR2 is :  0.9991619011985774
Testing has been finished!
The Total Testing Time is :  0.005318  seconds
Testing RMSE is :  [[0.01699925]]
Testing MAPE is :  [[0.00697883]]
Testing RR2 is :  0.9751179829431414
Training has been finished!
The Total Training Time is :  0.095745  seconds
Training RMSE is :  [[0.00391375]]
Training MAPE is :  [[0.00605696]]
Training RR2 is :  0.998819891337544
Testing has been finished!
The Total Testing Time is :  0.006913  seconds
Testing RMSE is :  [[0.00400868]]
Testing MAPE is :  [[0.00599674]]
Testing RR2 is :  0.9987332771010375
Training has been finished!
The Total Training Time is :  0.098735  seconds
Training RMSE is :  [[0.00363089]]
Training MAPE is :  [[0.00515497]]
Training RR2 is :  0.9989685055495083
Testing has been finished!
The Total Testing Time is :  0.004897  seconds
Testing RMSE is :

Training has been finished!
The Total Training Time is :  0.129652  seconds
Training RMSE is :  [[0.00208042]]
Training MAPE is :  [[0.00302422]]
Training RR2 is :  0.9996693074760687
Testing has been finished!
The Total Testing Time is :  0.004819  seconds
Testing RMSE is :  [[0.00238315]]
Testing MAPE is :  [[0.00325308]]
Testing RR2 is :  0.9995259543888172
Training has been finished!
The Total Training Time is :  0.129639  seconds
Training RMSE is :  [[0.00250393]]
Training MAPE is :  [[0.00395328]]
Training RR2 is :  0.9995152838235171
Testing has been finished!
The Total Testing Time is :  0.0054  seconds
Testing RMSE is :  [[0.00290712]]
Testing MAPE is :  [[0.00433486]]
Testing RR2 is :  0.9993633648547625
Training has been finished!
The Total Training Time is :  0.126696  seconds
Training RMSE is :  [[0.00308439]]
Training MAPE is :  [[0.00518017]]
Training RR2 is :  0.9992613248977663
Testing has been finished!
The Total Testing Time is :  0.005983  seconds
Testing RMSE is : 

Training has been finished!
The Total Training Time is :  0.110763  seconds
Training RMSE is :  [[0.00036432]]
Training MAPE is :  [[0.00056792]]
Training RR2 is :  0.9999898302285906
Testing has been finished!
The Total Testing Time is :  0.004932  seconds
Testing RMSE is :  [[0.00060898]]
Testing MAPE is :  [[0.00062259]]
Testing RR2 is :  0.9999701558529802
Training has been finished!
The Total Training Time is :  0.092753  seconds
Training RMSE is :  [[0.00076782]]
Training MAPE is :  [[0.00144795]]
Training RR2 is :  0.9999548526086205
Testing has been finished!
The Total Testing Time is :  0.003989  seconds
Testing RMSE is :  [[0.00076609]]
Testing MAPE is :  [[0.00144999]]
Testing RR2 is :  0.9999525320666142
Training has been finished!
The Total Training Time is :  0.100701  seconds
Training RMSE is :  [[0.0005447]]
Training MAPE is :  [[0.00098216]]
Training RR2 is :  0.9999770182661835
Testing has been finished!
The Total Testing Time is :  0.004989  seconds
Testing RMSE is :

In [54]:
 a=23.54
b=int(a)
print(b)    #18 14 23

23
